# Demo - Constraint Based Synthesis

Dependencies
<!-- - OPTIONAL: [Docker](https://www.docker.com/) -->
- Python 3ish
- [Jupyter](https://jupyter.org/)
- [Racket](https://racket-lang.org/)
- [Rosette](https://docs.racket-lang.org/rosette-guide/index.html): constraint based solver based on racket

Setup (No Docker)
1. Download & install [Racket](https://racket-lang.org/), add to path
2. Run:
    ```sh
    # optional but recommended: virtualenv
    python3 -m venv .venv
    source .venv/bin/activate
    pip install -r requirements.txt  # this installs jupyter

    # install rosette
    raco pkg install rosette

    jupyter notebook
    ```
3. Change environment variables at the top of your target notebook as required, as you may need to add racket to the venv path manually

In [ ]:
# Julia -- this is for my machine
import os

# Add /Applications/Racket v8.16/bin to PATH
os.environ['PATH'] = '/Applications/Racket v8.16/bin:$PATH'
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:$DYLD_LIBRARY_PATH'

# Example 1
## _Linear Function Synthesis from Examples_

Find a function of the form $f(x) = Ax + B$ that satisfies the following examples: 
- $f(1) = 5$
- $f(2) = 7$
- $f(3) = 9$
- $f(4) = 11$

---
```racket
#lang rosette
(require rosette/lib/synthax) ; the package containing `synthesis` syntax

; Define a function `f(x) = Ax + B`, with unknown coefficients `A` and `B` as `??`
(define (f x)
  (+ (* (?? integer?) x) (?? integer?)))  ; `??` means a hole in the program

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5)  (2 . 7)  (3 . 9)  (4 . 11)))

; Synthesis constraint: f(x) must match expected output
(define sol
  (synthesize
   #:forall (list)
   #:guarantee
   (begin
     (for/list ([pair examples])
       (assert (= (f (car pair)) (cdr pair))))))) ; for every pair (x, y) in `examples` assert f(x) = y

(print-forms sol)
```


In [ ]:
!racket racket/linear-func-synth.rkt

__Example 1 - Result:__   $f(x) = 2x + 3$

In [ ]:
# Example 1 -- Verbose Mode
!racket racket/linear-func-synth-VERBOSE.rkt

---
---

# Example 2 

Let's make this slightly more complicated and create a DSL (domain-specific language) for arithmetic over integers that includes addition, multiplication, and squaring.

--- 

```racket
#lang rosette

(require rosette/lib/synthax) ; the package containing `synthesis` syntax
(require rosette/lib/destruct)

; Let's define a simple grammar for arithmetic expressions over integers
; with addition,  multiplication, and squaring
(struct Add  (left right) #:transparent)
(struct Mult (left right) #:transparent)
(struct Square (arg) #:transparent)

(define (interpret expr)
  (destruct expr
    [(Add a b)    (+ (interpret a) (interpret b))]
    [(Mult a b)   (* (interpret a) (interpret b))]
    [(Square a)   (expt (interpret a) 2)]
    [_ expr]))


; Define a function `f(x) = Ax + B`, with unknown coefficients `A` and `B` as `??`
(define (f x)
  (Add (Mult (?? integer?) x) (?? integer?)))  ; `??` means a hole in the program

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5)  (2 . 7)  (3 . 9)  (4 . 11)))

; Synthesis constraint: f(x) must match expected output
(define sol
  (synthesize
   #:forall (list)
   #:guarantee
   (begin
     (for/list ([pair examples])
       (assert (= (interpret (f (car pair))) (cdr pair)))))))  ; for every pair (x, y) in `examples` assert f(x) = y

(print-forms sol)
```


In [ ]:
!racket racket/arith-dsl-1.rkt

__Example 2 - Result:__   $f(x) = 2x + 3$

In [ ]:
# Example 2 -- Verbose Mode
!racket racket/arith-dsl-1-VERBOSE.rkt

---
---

# Example 3

Using the same DSL, synthesize a quadratic expression of the form $Ax^2 + B$ from examples $f(2) = 9, f(3) = 19$.


---

```racket
; ... [same DSL code] ...

; Define a function f(x) = A * x^2 + B
(define (f x) (Add (Mult (?? integer?) (Square x)) (?? integer?))) 

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5) (2 . 9)  (3 . 19)))

; ...
```

In [ ]:
!racket racket/arith-dsl-2.rkt

__Example 3 - Result__: $f(x) = 2x^2 + 1$

In [ ]:
# Example 3 -- Verbose mode
!racket racket/arith-dsl-2-VERBOSE.rkt

---
--- 

# Example 4

Let's experiment with more complicated **sketching**. In previous examples, we synthesized functions of a certain forms (i.e. $f(x) = Ax + ...$), but since we defined this form explicitly for the solver, we only _really_  synthesized integer coefficients.

Before, we used `??` to represent integer "holes" that the solver should synthesize. 

Here we define a custom "hole" `??expr` to represent unknown arithmetic expressions of the following forms:

- $a + b$, $a*b$, $a$,  for some integer terminals $a, b$.
- _Note:_  this doesn't currently include $a + (a + b)$ and $a^2$


**Task:** Find two expressions $f, g$ such that $f + g = 10x$




---

```racket
; ... [same DSL code] ...

; In order to do `sketching`, we need to define constraints on an unknown expression `??expr` that we want to synthesize

(define (??expr terminals)
  (define l (apply choose* terminals))       ; given a list of args, `choose*` returns a value that can evaluate to any of them
  (define r (apply choose* terminals))       
  (choose* (Add l r) (Mult l r) l))

;  For instance, `??expr (list 2 x))` could return (Add 2 x), (Mult 2 x),  2, or x


; Use sketching to synthesize an expression of the form `f + g` equal to `10x`
(define-symbolic x c1 c2 integer?)  
(define sketch (Add (??expr (list x c1 c2)) (??expr (list x c1 c2))))

(define M      ; model
  (synthesize
    #:forall (list x)
    #:guarantee (assert (= (interpret sketch)
                           (interpret (Mult 10 x))))))
(evaluate sketch M)

```

In [ ]:
!racket racket/arith-dsl-sketch.rkt

__Example 4 - Result:__ 

$f := 15x,~ g := -5x ~\longrightarrow~ 15x + -5x = 10x$

&nbsp;

Bonus: the result changes if we change the order of the `choose*` statements in the source code.

In [ ]:
# Example 4 -- Verbose mode
!racket racket/arith-dsl-sketch-VERBOSE.rkt